# Training and evaluation of models

This notebook brings everything togehter to let us train and evaluate models in a simple manner. Due to no local GPU resources, this notebook contains code for Google Colab, so it **must be executed in Google Colab**.

<a target="_blank" href="https://colab.research.google.com/github/Moikii/SHEL">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

After downloading an existing dataset, or generating a new one (which takes more time for a large number of images), we can train a model with some custom parameters. Evaluation follows next, where different models performance can be compared.

In [2]:
!rm -rf sample_data
!git clone https://github.com/Moikii/SHEL.git

Cloning into 'SHEL'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 91 (delta 38), reused 64 (delta 16), pack-reused 0
Receiving objects: 100% (91/91), 12.46 MiB | 14.91 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [3]:
%%capture
!pip install ultralytics

import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from IPython.utils import io
from ultralytics import YOLO
py_file_location = "/content/SHEL/src"
sys.path.append(os.path.abspath(py_file_location))
import dataset_generation_functions as dgf

In [4]:
# options : 'new_dataset', 'load_dataset', 'load_model'
starting_point = 'load_model'

# only when creating a new dataset
number_of_images = 100
max_number_of_cards_per_image = 4
min_size = 0.2
max_size = 0.7
max_rotation = 360
overlapping = True
seed = 42

# model training parameters
epochs = 3

In [ ]:
if starting_point == 'new_dataset':
  !rm -rf dataset
  !. /content/SHEL/src/drive_data_download '1KHAYHo9voKRUWtff5ES07hBRUmT7vSg5' 'backgrounds'
  !. /content/SHEL/src/drive_data_download '1o0f3q5zJzfRXs9vSMb1uR9KcUfWtFItZ' 'photos'
  DATASET_DIR = dgf.generate_dataset('/content/backgrounds', '/content/photos', '/content/dataset', number_of_images,
                                     max_number_of_cards_per_image, min_size, max_size, max_rotation, overlapping, seed)
  !rm -rf photos photos_processed backgrounds
  model = YOLO('yolov8n.pt')
  results = model.train(data = DATASET_DIR + '/data.yaml', epochs = epochs)
elif starting_point == 'load_dataset':
  !rm -rf dataset
  !. /content/SHEL/src/drive_data_download '11cMzPUd_ETyOzzb0iJMtONWP10Nu7Zwc' 'dataset'
  print('Downloading dataset from external source...')
  DATASET_DIR = '/content/dataset'
  model = YOLO('yolov8n.pt')
  results = model.train(data = DATASET_DIR + '/data.yaml', epochs = epochs)
elif starting_point == 'load_model':
  !rm -rf dataset
  !. /content/SHEL/src/drive_data_download '11cMzPUd_ETyOzzb0iJMtONWP10Nu7Zwc' 'dataset'
  !. /content/SHEL/src/drive_data_download '1DJ2CTQTFUvKljhlD2Kn9X0tpTOx8w8cG' 'runs'
else: print('Invalid input!')

In [28]:
precisions = list()
recalls = list()
for folder in next(os.walk('/content/runs/detect'))[1]:
  best_model_path = '/content/runs/detect/' + folder + '/weights/best.pt'
  training_results_df = pd.read_csv('/content/runs/detect/' + folder + '/results.csv', usecols=[0,4,5], names = ('epoch', 'precision', 'recall'),
                                  skipinitialspace=True, skiprows = 1, dtype={'recall': float, 'precision': float})
  model = YOLO(best_model_path)

  precisions.append(training_results_df['precision'].tolist())
  recalls.append(training_results_df['recall'].tolist())

  print(f'Results of model in "{best_model_path}":')
  print('-----------------------')
  print(f'Number of epochs: {training_results_df["epoch"].iloc[-1]}')
  print(f'Mean recall at end of training: {training_results_df["recall"].iloc[-1]}')
  print(f'Mean precision at end of training: {training_results_df["precision"].iloc[-1]}')


Results of model in "/content/runs/detect/train/weights/best.pt":
-----------------------
Number of epochs: 2
Mean recall at end of training: 0.92616
Mean precision at end of training: 0.97942


In [13]:
#!rm -rf runs
#